# Reelgood Scraper

Workflow: 3  

Goal: Start with the list of reelgood movie urls and get the data for each movie. Split the urls into jobs. 

Result: The file ```reelgood_movie.csv``` is created.

In [1]:
import os
import time
import pandas as PD

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
# can't reload script in a different folder

import sys
sys.path.append('../models')
import reelgood_scraper as RS 

## Create Scraping Jobs 

In [4]:
URL_FILE = '../data/reelgood_url.csv'
url_df = PD.read_csv(URL_FILE)
url_df.head()
url_df.info()

,title,year,reelgood_id
0,1,2013,1-2013
1,"10,000 BC",2008,10000-bc-2008
2,1000 Rupee Note,2016,1000-rupee-note-2016
3,1000 To 1,2014,1000-to-1-2014
4,$100 and a T-Shirt: A Documentary About Zines ...,2004,100-and-a-tshirt-a-documentary-about-zines-in-...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18496 entries, 0 to 18495
Data columns (total 3 columns):
title          18496 non-null object
year           18496 non-null int64
reelgood_id    18496 non-null object
dtypes: int64(1), object(2)
memory usage: 433.6+ KB


In [5]:
url_1_df = url_df[0:3054]
url_2_df = url_df[3055:8999]
url_3_df = url_df[9000:13499]
url_4_df = url_df[13500:]

url_1_df.shape
url_2_df.shape
url_3_df.shape
url_4_df.shape

(4499, 3)

(4499, 3)

(4499, 3)

(4996, 3)

## Scrape Rotation

In [6]:
scraper = RS.ReelgoodScraper()
scraper.log_in()

<selenium.webdriver.firefox.webdriver.WebDriver (session="1aa39c5a-add6-ec4b-b235-28330e2dbb04")>
https://reelgood.com/?login=true


In [7]:
t0 = time.time()
movie_ls = []

for idx, row in url_2_df.iterrows():
    if idx % 500 == 0:
        print(f'row: {idx}')
        
    rg_id = row['reelgood_id']
    movie_dx = scraper.get_movie_data(rg_id)
    movie_ls.append(movie_dx)

scraper.close()
t1 = time.time()
print(f'time: {(t1-t0)/60/60:.2f} hrs')

row: 0
row: 500
row: 1000
row: 1500
row: 2000
row: 2500
row: 3000


TimeoutException: Message: Timeout loading page after 300000ms


In [10]:
movie_df = PD.DataFrame(movie_ls)
movie_df.head()
movie_df.info()

,title,year,duration,rating,country,poster,genres,tags,imdb_score,rt_score,synopsis,services
0,1,2013,112.0,13+ (PG-13),America,https://img.reelgood.com/content/movie/9b61df6...,"Documentary, Mystery, Thriller, Science-Fictio...","Car, Racing",8,None,Set in the golden era of Grand Prix Racing '1'...,"{""reelgood"": ""1-2013"", ""amazon"": ""B0751Q3J8P""}"
1,"10,000 BC",2008,109.0,13+ (PG-13),South Africa,https://img.reelgood.com/content/movie/84ff615...,"Action & Adventure, Drama, Fantasy, History","Egypt, Animal, Fighting, Hunting",5.1,8%,A prehistoric epic that follows a young mammot...,"{""reelgood"": ""10000-bc-2008"", ""netflix"": ""7006..."
2,1000 Rupee Note,2016,89.0,None,None,https://img.reelgood.com/content/movie/08248c1...,Drama,None,7.3,80%,Poor Parobudhi receives a thousand rupee note ...,"{""reelgood"": ""1000-rupee-note-2016"", ""netflix""..."
3,1000 To 1,2014,99.0,None,None,https://img.reelgood.com/content/movie/dcc9187...,"Drama, Biography",None,6.9,None,Cory Weissman is a college basketball player w...,"{""reelgood"": ""1000-to-1-2014"", ""amazon"": ""B07D..."
4,$100 and a T-Shirt: A Documentary About Zines ...,2004,51.0,None,None,https://img.reelgood.com/content/movie/d2a26a5...,Documentary,None,6.3,None,A cultural analysis of what causes zine maker...,"{""reelgood"": ""100-and-a-tshirt-a-documentary-a..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3056 entries, 0 to 3055
Data columns (total 12 columns):
title         3056 non-null object
year          3056 non-null object
duration      2997 non-null float64
rating        1391 non-null object
country       1187 non-null object
poster        3056 non-null object
genres        2951 non-null object
tags          1348 non-null object
imdb_score    2973 non-null object
rt_score      863 non-null object
synopsis      3056 non-null object
services      3056 non-null object
dtypes: float64(1), object(11)
memory usage: 286.6+ KB


In [13]:
save_path = r'reelgood_mov_1.csv'
movie_df.to_csv(save_path, index=False)
f'{round(os.path.getsize(save_path) /1e6, 2)} mb'

'1.71 mb'

In [16]:
movie_df.to_csv(index=False)

'title,year,duration,rating,country,poster,genres,tags,imdb_score,rt_score,synopsis,services\n1,2013,112.0,13+ (PG-13),America,https://img.reelgood.com/content/movie/9b61df62-90bc-457f-b482-b1bd6ff874a8/poster-780.jpg,"Documentary, Mystery, Thriller, Science-Fiction, History, Sport","Car, Racing",8,,"Set in the golden era of Grand Prix Racing \'1\' tells the story of a generation of charismatic drivers who raced on the edge, risking their lives during Formula 1\'s deadliest period, and the men who stood up and changed the sport forever.","{""reelgood"": ""1-2013"", ""amazon"": ""B0751Q3J8P""}"\n"10,000 BC",2008,109.0,13+ (PG-13),South Africa,https://img.reelgood.com/content/movie/84ff6154-b1bd-4aee-abf3-7eb77900d096/poster-780.jpg,"Action & Adventure, Drama, Fantasy, History","Egypt, Animal, Fighting, Hunting",5.1,8%,A prehistoric epic that follows a young mammoth hunter\'s journey through uncharted territory to secure the future of his tribe.,"{""reelgood"": ""10000-bc-2008"", ""netfl

## Merge Job Files

In [ ]:
st op = 
movie_1_df = PD.read_csv('../data/reelgood_mov_1.csv')
movie_2_df = PD.read_csv('../data/reelgood_mov_2.csv')
movie_3_df = PD.read_csv('../data/reelgood_mov_3.csv')
movie_4_df = PD.read_csv('../data/reelgood_mov_4.csv')

movie_1_df.shape
movie_2_df.shape
movie_3_df.shape
movie_4_df.shape

In [ ]:
# remove duplicates



In [ ]:
movie_df = PD.concat([movie_1_df, movie_2_df, movie_3_df, movie_4_df])
movie_df.head()
movie_df.info()

In [ ]:
save_path = r'../data/reelgood_movie.csv'
movie_df.to_csv(save_path, index=False)
f'{round(os.path.getsize(save_path) /1e6, 2)} mb'